In [1]:
import pandas as pd
import numpy as np
df = []
# df = pd.read_excel('Pos Tag with Opinion Terms.xlsx')
df = pd.read_excel('/content/UC_Dataset.xlsx')
df_check = df['Sentences'].head(20)

In [2]:
df.columns

Index(['Sentences', 'OpinionTerm1', 'Aspect Category1', 'AspectPolarity1 ',
       'OpinionTerm2', 'Aspect Category2', 'Aspectpolarity2'],
      dtype='object')

In [3]:
df.head()

,Sentences,OpinionTerm1,Aspect Category1,AspectPolarity1,OpinionTerm2,Aspect Category2,Aspectpolarity2
0,excellent service....very professional electri...,excellent service,Company_Service,positive,professional electricians,service_person_attitude,positive
1,nice and good app,good app,App_Utility,positive,NaN,NaN,NaN
2,came on time work proeffetionaly complete,on time,service_schedule,positive,work proeffetionaly,service_person_attitude,positive
3,first time user and very satisfied.,first time user,App_Functionality,positive,satisfied,Company_Service,positive
4,nice facilities and rates is to much high,nice facilities,Company_Service,positive,much high,cost,negative


###  Spiltting Rules  rules

In [5]:
import re
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd  # Import the pandas library
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

conjunction_preposition = ['and', 'or', 'with', 'but', 'by', 'for', 'at', 'in']
helping_verbs = ["is", "am", "are", "was", "were", "have", "has", "had", "do", "does", "did", "can", "could", "will", "would",
                 "shall", "should", "may", "might", "must"]

def apply_rules(sentence):
    sentence = sentence.replace('&', 'and').replace('..', '.').replace(',,', ',').replace('  ', ' ').strip('.').strip(
        ',').strip().strip('...').strip('..').strip(',,')
    tokens = word_tokenize(sentence)

    # Rule #1: Split by '.', ',' (give priority to ',')
    if len(tokens) > 4:

        # Rule #A: Punctuation followed by punctuation
        if re.search(r'[,\.]', sentence):
            split_sentences = re.split(r'[,\.]', sentence, 1)
            return split_sentences, 'Rule #A (1.1)'

        elif re.search(r'\.\.', sentence):
            split_sentences = re.split(r'\.\.', sentence, 1)
            return split_sentences, 'Rule #A (1.2)'

        # Rule #B: Punctuation followed by conjunction
        for cp in conjunction_preposition:
            if cp in tokens and re.search(r'[,\.]', sentence):
                split_sentences = re.split(r'(?<!\S)\b{}\b(?!\S)'.format(cp), sentence, 1)
                return split_sentences, 'Rule #B (1.1)'

        # Rule #C: Helping Verb followed by punctuation
        for hv in helping_verbs:
            if hv in tokens and re.search(r'[,\.]', sentence):
                split_sentences = re.split(r'(?<!\S)\b{}\b(?!\S)'.format(hv), sentence, 1)
                return split_sentences, 'Rule #C (1.1)'

        # Rule #D: Conjunction followed by punctuation
        for cp in conjunction_preposition:
            if cp in tokens and re.search(r'[,\.]', sentence):
                # Skip 'and' conjunction
                if cp == 'and':
                    continue
                split_sentences = re.split(r'(?<!\S)\b{}\b(?!\S)'.format(cp), sentence, 1)
                return split_sentences, 'Rule #D'

    # Rule #2: Split by conjunctions or prepositions
    if len(tokens) > 4:
        for cp in conjunction_preposition:
            if cp in tokens:
                split_sentences = re.split(r'(?<!\S)\b{}\b(?!\S)'.format(cp), sentence, 1)
                if len(split_sentences) == 2:
                    return split_sentences, 'Rule #2'

    # Rule #3: Split by noun case
    if len(tokens) > 4:
        pos_tags = nltk.pos_tag(tokens)
        for i, (_, pos) in enumerate(pos_tags[1:], start=1):
            if pos.startswith('NN'):
                split_sentences = [' '.join(tokens[:i+1]), ' '.join(tokens[i+1:])]
                return split_sentences, 'Rule #3'

    return [sentence], 'No rule applied'

# Create a list of sentences to process, e.g., df_check
#df_check = ["This is a test sentence.", "I like apples and oranges.", "She can dance, sing, and play the guitar."]

# Create an empty list to store the results
results = []

# Applying the rules to each sentence
for sentence in df_check:
    broken_sentences, rule_applied = apply_rules(sentence)
    results.append([sentence, broken_sentences, rule_applied])

# Create a DataFrame from the results list
df_results = pd.DataFrame(results, columns=['Original Sentence', 'Broken Sentences', 'Rule Applied'])

# Save the DataFrame to an Excel file
output_file = 'broken_sentences.xlsx'  # Choose a file name for your output Excel file
df_results.to_excel(output_file, index=False)  # Save to Excel without an index column


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Implementing this spilting fucntion to dataframe senteces and making 2 new columns "Broken Senteces" , "Split Rule# Number"

In [6]:
df['Broken Sentences'], df['Split Rule# Number'] = zip(*df['Sentences'].map(apply_rules))

### Sample code of Extraction of Opinion  Term

## code Opinion term rules

In [8]:
import re
import nltk
import pandas as pd

def extract_patterns(text):
    # Define the patterns, their respective regex patterns, and priorities
    patterns = [
        ('JJ,NN', re.compile(r'(\w+),(\w+) (NN)\b'), 1),
        ('NNP,NN', re.compile(r'(\w+),(\w+) (NN)(?!P)\b'), 2),
        ('JJ,NNS', re.compile(r'(\w+),(\w+) (JJ),(NNS)\b'), 3),
        ('NN,NN', re.compile(r'(\w+),(\w+) (NN),(NN)\b'), 4),
        ('NNP,NNP', re.compile(r'(\w+),(\w+) (NNP),(NNP)\b'), 5),
        ('RB,JJ', re.compile(r'(\w+),(\w+) (RB),(JJ)\b'), 6),
        ('RB,NN', re.compile(r'(\w+),(\w+) (RB),(NN)\b'), 7),
        ('NN,VBZ,JJ', re.compile(r'(\w+),(\w+) (NN),(VBZ),(JJ)\b'), 8),
        ('NN,NNS', re.compile(r'(\w+),(\w+) (NN),(NNS)\b'), 9),
        ('NN,JJ', re.compile(r'(\w+),(\w+) (NN),(JJ)\b'), 10),
        ('CD,NN', re.compile(r'(\w+),(\w+) (CD),(NN)\b'), 11),
        ('VBN,NN', re.compile(r'(\w+),(\w+) (VBN),(NN)\b'), 12),
        ('JJR,NNS', re.compile(r'(\w+),(\w+) (JJR),(NNS)\b'), 13),
        ('JJ,NN,NN', re.compile(r'(\w+),(\w+),(\w+) (JJ),(NN),(NN)\b'), 14),
        ('JJS,NN', re.compile(r'(\w+),(\w+) (JJS),(NN)\b'), 15),
        ('NNP,NN,NN', re.compile(r'(\w+),(\w+),(\w+) (NNP),(NN),(NN)\b'), 16),
        ('RB,VBN', re.compile(r'(\w+),(\w+) (RB),(VBN)\b'), 17),
        ('NN', re.compile(r'(\w+) (NN)\b'), 18),
        ('NNS', re.compile(r'(\w+) (NNS)\b'), 19),
        ('JJ', re.compile(r'(\w+) (JJ)\b'), 20),
    ]

    matches = re.findall(r'\b\w+\b', text)
    patterns_and_instances = []

    for pattern, regex, priority in patterns:
        instances = []
        for i in range(1, len(matches)):
            match = regex.match(f'{matches[i-1]} {matches[i]}')
            if match:
                instances.append((matches[i-1], matches[i]))
        if instances:
            patterns_and_instances.append((pattern, instances, priority))

    patterns_and_instances.sort(key=lambda x: x[2])
    return patterns_and_instances

# Function to match patterns and retrieve corresponding text
def match_patterns(tags, words, patterns):
    matched_texts = {}
    for pattern in patterns:
        pattern_tags = pattern.split(',')
        matched_text = []
        for i in range(len(tags) - len(pattern_tags) + 1):
            if all(tags[i+j] == pattern_tags[j] for j in range(len(pattern_tags))):
                matched_text.append(' '.join(words[i:i+len(pattern_tags)]))
        matched_texts[pattern] = matched_text
    return matched_texts


# Example usage on DataFrame


def predict_opinion_terms(sentences):
    predicted_opinion_terms = []
    for sentence in sentences:
        if not sentence:  # Skip empty sentences
            continue
        words = nltk.word_tokenize(sentence.strip().strip('.').strip().lower())
        pos_tags = nltk.pos_tag(words)
        tags = [tag for word, tag in pos_tags]
        tags_list = [x[1] for x in pos_tags]
        tags_extract_patterns = ' '.join(tags_list)
        patterns = extract_patterns(tags_extract_patterns)
        if not patterns:  # Skip when no pattern is found
            predicted_opinion_terms.append(None)
            continue
        patterns1 = [','.join(patterns[0][1][0])]
        matched_texts = match_patterns(tags, words, patterns1)
        values_list = list(matched_texts.values())
        try:
            sentence_opinion_term = values_list[0][0]
        except:
            sentence_opinion_term = ''
        predicted_opinion_terms.append(sentence_opinion_term)
    return predicted_opinion_terms

def apply_prediction(x):
    try:
        prediction = predict_opinion_terms(x)[0] if x else np.nan
        return prediction
    except Exception as e:
        print(f"Prediction failed for sentence: '{x}'. Error: {e}")
        return np.nan



df['Predicted Opinion Term 1'] = df['Broken Sentences'].apply(apply_prediction)
#df['Predicted Opinion Term 1'] = df['Broken Sentences'].apply(lambda x: predict_opinion_terms(x)[0] if x else np.nan)
df['Predicted Opinion Term 2'] = df['Broken Sentences'].apply(lambda x: predict_opinion_terms(x)[1] if len(predict_opinion_terms(x)) > 1 else np.nan)

In [9]:
df

,Sentences,OpinionTerm1,Aspect Category1,AspectPolarity1,OpinionTerm2,Aspect Category2,Aspectpolarity2,Broken Sentences,Split Rule# Number,Predicted Opinion Term 1,Predicted Opinion Term 2
0,excellent service....very professional electri...,excellent service,Company_Service,positive,professional electricians,service_person_attitude,positive,"[excellent service, .very professional electri...",Rule #A (1.1),excellent service,professional electrician
1,nice and good app,good app,App_Utility,positive,NaN,NaN,NaN,[nice and good app],No rule applied,good app,NaN
2,came on time work proeffetionaly complete,on time,service_schedule,positive,work proeffetionaly,service_person_attitude,positive,"[came on time, work proeffetionaly complete]",Rule #3,on time,work proeffetionaly
3,first time user and very satisfied.,first time user,App_Functionality,positive,satisfied,Company_Service,positive,"[first time user , very satisfied]",Rule #2,first time,very satisfied
4,nice facilities and rates is to much high,nice facilities,Company_Service,positive,much high,cost,negative,"[nice facilities , rates is to much high]",Rule #2,nice facilities,to much
...,...,...,...,...,...,...,...,...,...,...,...
4159,good service,good service,Company_Service,positive,NaN,NaN,NaN,[good service],No rule applied,good service,NaN
4160,Really impressive service from welltrained pro...,impressive service,Company_Service,positive,welltrained professionals,service_person_attitude,Positive,"[Really impressive service, from welltrained p...",Rule #3,impressive service,welltrained professionals
4161,very good app,good app,App_Utility,positive,NaN,NaN,NaN,[very good app],No rule applied,good app,NaN
4162,service not good no analysis worst service,service not good,Company_Service,positive,NaN,NaN,NaN,"[service not good no analysis, worst service]",Rule #3,no analysis,worst service


In [10]:
df.columns

Index(['Sentences', 'OpinionTerm1', 'Aspect Category1', 'AspectPolarity1 ',
       'OpinionTerm2', 'Aspect Category2', 'Aspectpolarity2',
       'Broken Sentences', 'Split Rule# Number', 'Predicted Opinion Term 1',
       'Predicted Opinion Term 2'],
      dtype='object')

In [11]:
df['OpinionTerm1'] = df['OpinionTerm1'].apply(lambda x: x.strip().lower() if isinstance(x, str) else np.nan)
df['OpinionTerm2'] = df['OpinionTerm2'].apply(lambda x: x.strip().lower() if isinstance(x, str) else np.nan)

df['Predicted Opinion Term 1'] = df['Predicted Opinion Term 1'].apply(lambda x: x.strip().lower() if isinstance(x, str) else np.nan)
df['Predicted Opinion Term 2'] = df['Predicted Opinion Term 2'].apply(lambda x: x.strip().lower() if isinstance(x, str) else np.nan)

In [12]:
# Define a custom function to handle comparisons
def custom_comparison1(row):
    if pd.isnull(row['OpinionTerm1']) and pd.isnull(row['Predicted Opinion Term 1']):
        return True
    return row['OpinionTerm1'] == row['Predicted Opinion Term 1']

In [13]:
# Define a custom function to handle comparisons
def custom_comparison2(row):
    if pd.isnull(row['OpinionTerm2']) and pd.isnull(row['Predicted Opinion Term 2']):
        return True
    return row['OpinionTerm2'] == row['Predicted Opinion Term 2']

In [14]:
#df['Comparison 1'] = df['Opinion Term '] == df['Predicted Opinion Term 1']
df['Comparison 1'] = df.apply(custom_comparison1, axis=1)
accuracy = df['Comparison 1'].value_counts(normalize=True) * 100
true_cases = df['Comparison 1'].value_counts()[True]
false_cases = df['Comparison 1'].value_counts()[False]

print(f"\nAccuracy: {accuracy[True]:.2f}%")
print(f"True Cases: {true_cases}")
print(f"False Cases: {false_cases}")


Accuracy: 81.44%
True Cases: 3391
False Cases: 773


In [15]:
import pandas as pd

# Assuming you have a DataFrame 'df' with columns 'OpinionTerm1' and 'Predicted Opinion Term 1'

def custom_comparison1(row):
    if pd.isnull(row['OpinionTerm1']) and pd.isnull(row['Predicted Opinion Term 1']):
        return True
    return row['OpinionTerm1'] == row['Predicted Opinion Term 1']

df['Comparison 1'] = df.apply(custom_comparison1, axis=1)

true_cases = df['Comparison 1'].sum()  # Count of True values in 'Comparison 1' column
false_cases = len(df) - true_cases  # Count of False values in 'Comparison 1' column

accuracy = (true_cases / len(df)) * 100

print(f"\nAccuracy: {accuracy:.2f}%")
print(f"True Cases: {true_cases}")
print(f"False Cases: {false_cases}")



Accuracy: 81.44%
True Cases: 3391
False Cases: 773


In [16]:


def custom_comparison1(row):
    if pd.isnull(row['OpinionTerm1']) and pd.isnull(row['Predicted Opinion Term 1']):
        return True
    return row['OpinionTerm1'] == row['Predicted Opinion Term 1']

# Calculate TP, TN, FP, FN
df['TP'] = df.apply(lambda row: 1 if custom_comparison1(row) else 0, axis=1)
df['TN'] = df.apply(lambda row: 1 if pd.isnull(row['OpinionTerm1']) and pd.isnull(row['Predicted Opinion Term 1']) else 0, axis=1)
df['FP'] = df.apply(lambda row: 1 if not pd.isnull(row['OpinionTerm1']) and (pd.isnull(row['Predicted Opinion Term 1']) or row['OpinionTerm1'] != row['Predicted Opinion Term 1']) else 0, axis=1)
df['FN'] = df.apply(lambda row: 1 if pd.isnull(row['OpinionTerm1']) and not pd.isnull(row['Predicted Opinion Term 1']) else 0, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = df['TP'].sum() / (df['TP'].sum() + df['FP'].sum())
recall = df['TP'].sum() / (df['TP'].sum() + df['FN'].sum())
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)




Precision: 0.8145568099927937
Recall: 0.9997051886792453
F1 Score: 0.8976836532097949


In [17]:
#df['Comparison 2'] = df['opinion term'] == df['Predicted Opinion Term 2']
df['Comparison 2'] = df.apply(custom_comparison2, axis=1)
accuracy = df['Comparison 2'].value_counts(normalize=True) * 100
true_cases = df['Comparison 2'].value_counts()[True]
false_cases = df['Comparison 2'].value_counts()[False]

print(f"\nAccuracy: {accuracy[True]:.2f}%")
print(f"True Cases: {true_cases}")
print(f"False Cases: {false_cases}")


Accuracy: 75.82%
True Cases: 3157
False Cases: 1007


In [18]:

def custom_comparison1(row):
    if pd.isnull(row['OpinionTerm2']) and pd.isnull(row['Predicted Opinion Term 2']):
        return True
    return row['OpinionTerm2'] == row['Predicted Opinion Term 2']

# Calculate TP, TN, FP, FN
df['TP'] = df.apply(lambda row: 1 if custom_comparison1(row) else 0, axis=1)
df['TN'] = df.apply(lambda row: 1 if pd.isnull(row['OpinionTerm2']) and pd.isnull(row['Predicted Opinion Term 2']) else 0, axis=1)
df['FP'] = df.apply(lambda row: 1 if not pd.isnull(row['OpinionTerm2']) and (pd.isnull(row['Predicted Opinion Term 2']) or row['OpinionTerm2'] != row['Predicted Opinion Term 2']) else 0, axis=1)
df['FN'] = df.apply(lambda row: 1 if pd.isnull(row['OpinionTerm2']) and not pd.isnull(row['Predicted Opinion Term 2']) else 0, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = df['TP'].sum() / (df['TP'].sum() + df['FP'].sum())
recall = df['TP'].sum() / (df['TP'].sum() + df['FN'].sum())
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Precision: 0.8933220147142049
Recall: 0.833641404805915
F1 Score: 0.8624504849064335


In [19]:
def calculate_comparison_3(row):
    if row['Comparison 1'] == True and row['Comparison 2'] == True:
        return True
    return False

In [20]:
#df['Comparison 3'] =  df['Comparison 2'] == True & df['opinion term'].eq(df['Predicted Opinion Term 2'])
df['Comparison 3'] = df.apply(calculate_comparison_3, axis=1)
accuracy = df['Comparison 3'].value_counts(normalize=True) * 100
true_cases = df['Comparison 3'].value_counts()[True]
false_cases = df['Comparison 3'].value_counts()[False]

print(f"\nAccuracy: {accuracy[True]:.2f}%")
print(f"True Cases: {true_cases}")
print(f"False Cases: {false_cases}")


Accuracy: 64.89%
True Cases: 2702
False Cases: 1462


In [21]:

def calculate_comparison_3(row):
    if row['Comparison 1'] == True and row['Comparison 2'] == True:
        return True
    return False

df['TP'] = df.apply(lambda row: 1 if calculate_comparison_3(row) else 0, axis=1)
df['TN'] = df.apply(lambda row: 1 if not row['Comparison 1'] and not row['Comparison 2'] else 0, axis=1)
df['FP'] = df.apply(lambda row: 1 if row['Comparison 1'] and not row['Comparison 2'] else 0, axis=1)
df['FN'] = df.apply(lambda row: 1 if not row['Comparison 1'] and row['Comparison 2'] else 0, axis=1)

# Calculate Precision, Recall, and F1 Score
precision = df['TP'].sum() / (df['TP'].sum() + df['FP'].sum())
recall = df['TP'].sum() / (df['TP'].sum() + df['FN'].sum())
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

Precision: 0.7968150987909172
Recall: 0.8558758314855875
F1 Score: 0.8252901649358584


In [22]:
df.columns

Index(['Sentences', 'OpinionTerm1', 'Aspect Category1', 'AspectPolarity1 ',
       'OpinionTerm2', 'Aspect Category2', 'Aspectpolarity2',
       'Broken Sentences', 'Split Rule# Number', 'Predicted Opinion Term 1',
       'Predicted Opinion Term 2', 'Comparison 1', 'TP', 'TN', 'FP', 'FN',
       'Comparison 2', 'Comparison 3'],
      dtype='object')

In [23]:
df1  = df.copy()
df1 = df1.reindex(columns=['Sentences','Broken Sentences', 'Split Rule# Number', 'Spilting',
        'OpinionTerm1','Predicted Opinion Term 1', 'Comparison 1', 'Aspect Category1',
       'AspectPolarity1 ', 'OpinionTerm2','Predicted Opinion Term 2','Comparison 2', 'Aspect Category2',
       'Aspectpolarity2', 'Token Count', 'Comparison 3'])